# 🌐 Week 8 — Introduction to Web Scraping

## 🎯 Session Overview

Welcome to **Web Scraping**! This is where Python becomes your data-gathering superpower. Web scraping allows you to automatically extract data from websites - think of it as teaching Python to read and collect information from web pages just like you do.

### What is Web Scraping?

```
┌─────────────────────────────────────────────────────────────────┐
│                     WEB SCRAPING WORKFLOW                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   🌐 Website  →  📥 Fetch HTML  →  🔍 Parse  →  📊 Extract     │
│                                                                 │
│   example.com     HTTP Request      HTML         Structured     │
│                   (urllib)          Parser       Data           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Real-World Applications

| Use Case | Example |
|----------|---------|
| 📊 **Data Collection** | Gathering product prices for comparison |
| 📰 **News Aggregation** | Collecting headlines from multiple sources |
| 🏠 **Real Estate** | Monitoring property listings |
| 💼 **Job Hunting** | Scraping job postings |
| 📈 **Research** | Collecting academic data |

### ⚠️ Important: Ethics & Legality

| Do ✅ | Don't ❌ |
|------|---------|
| Check robots.txt | Ignore rate limits |
| Respect Terms of Service | Scrape personal data |
| Add delays between requests | Overwhelm servers |
| Use APIs when available | Bypass authentication |
| Identify your bot politely | Claim to be a browser |

## 🎯 Learning Objectives

By the end of this session, you will be able to:

- ✅ **Fetch** web pages using Python's `urllib` standard library
- ✅ **Parse** HTML content using `html.parser` from the standard library
- ✅ **Extract** links and data from HTML documents
- ✅ **Handle** common web scraping errors and edge cases
- ✅ **Understand** ethical considerations and best practices
- ✅ **Apply** web scraping techniques to gather real data

### 📚 Topics Covered

| Section | Topic | Key Concept |
|---------|-------|-------------|
| 1 | HTTP Basics | How web requests work |
| 2 | Fetching Pages | urllib.request |
| 3 | Parsing HTML | html.parser |
| 4 | Extracting Data | Finding specific elements |
| 5 | Error Handling | Robust scraping |
| 6 | Best Practices | Ethical scraping |

---

### 🔄 JavaScript Transition Note

If you're coming from JavaScript/web development:

| JavaScript | Python | Purpose |
|------------|--------|---------|
| `fetch()` / `axios` | `urllib.request` | Make HTTP requests |
| `document.querySelector()` | `html.parser` | Parse HTML |
| `element.getAttribute()` | Handler methods | Extract attributes |
| `JSON.parse()` | `json.loads()` | Parse JSON responses |

**Key Differences:**
- Python's `html.parser` uses an **event-driven** approach (callbacks for tags)
- JavaScript's DOM is **tree-based** (query selectors)
- Both achieve the same goal: extracting data from HTML!

---

## 📡 Section 1: Understanding HTTP Basics

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📡 HTTP REQUEST BASICS
# ═══════════════════════════════════════════════════════════════════════════════
# Before we scrape, let's understand what happens when you visit a website:

print("""
┌────────────────────────────────────────────────────────────────────────────┐
│                        HTTP REQUEST FLOW                                   │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                            │
│  Your Code              Request               Web Server                   │
│  ┌─────────┐    ──────────────────────>      ┌─────────────┐              │
│  │ Python  │    GET /page HTTP/1.1           │   Server    │              │
│  │ Script  │    Host: example.com            │  (Apache,   │              │
│  │         │    User-Agent: Python           │   Nginx)    │              │
│  │         │                                 │             │              │
│  │         │    <──────────────────────      │             │              │
│  │         │    HTTP/1.1 200 OK              │             │              │
│  │         │    Content-Type: text/html      │             │              │
│  │         │    <html>...</html>             │             │              │
│  └─────────┘                                 └─────────────┘              │
│                                                                            │
└────────────────────────────────────────────────────────────────────────────┘
""")

print("📋 Common HTTP Status Codes:")
print("=" * 50)
status_codes = {
    200: ("OK", "Request successful"),
    301: ("Moved Permanently", "Page has moved"),
    403: ("Forbidden", "Access denied"),
    404: ("Not Found", "Page doesn't exist"),
    429: ("Too Many Requests", "Rate limited!"),
    500: ("Server Error", "Website problem")
}
for code, (name, desc) in status_codes.items():
    emoji = "✅" if code == 200 else "⚠️" if code < 500 else "❌"
    print(f"  {emoji} {code}: {name} - {desc}")

---

## 📥 Section 2: Fetching Web Pages with urllib

The `urllib` module is Python's built-in tool for making HTTP requests. No installation needed!

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚫 BAD PRACTICE: Fetching without error handling
# ═══════════════════════════════════════════════════════════════════════════════
# ❌ Problem: No error handling, no timeout, crashes on failure

from urllib.request import urlopen

# This will crash if the URL is invalid or network fails!
# response = urlopen("https://definitely-not-a-real-website-12345.com")

# Let's simulate with a simple HTML string instead
html = '<html><body><a href="https://example.com">example</a></body></html>'
print(f"HTML length: {len(html)} characters")

# ⚠️ What's wrong here:
# - No try/except for network errors
# - No timeout (could hang forever)
# - No status code checking
# - No User-Agent (might be blocked)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🌱 NOVICE: Basic fetch with error handling
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Better: Added try/except for errors

from urllib.request import urlopen
from urllib.error import URLError, HTTPError

def fetch_page(url):
    """Fetch a web page and return its content."""
    try:
        with urlopen(url) as response:
            return response.read().decode('utf-8')
    except HTTPError as e:
        print(f"HTTP Error: {e.code}")
        return None
    except URLError as e:
        print(f"URL Error: {e.reason}")
        return None

# Test with a sample (we'll use embedded HTML for safety)
sample_html = """
<!DOCTYPE html>
<html>
<head><title>Sample Page</title></head>
<body>
    <h1>Welcome!</h1>
    <a href="https://example.com">Example Link</a>
</body>
</html>
"""

print("✅ Function defined! Here's our sample HTML:")
print("=" * 50)
print(sample_html)
print("=" * 50)
print(f"Length: {len(sample_html)} characters")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 INTERMEDIATE: Fetch with timeout and headers
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Better: Added timeout, User-Agent header, status code checking

from urllib.request import urlopen, Request
from urllib.error import URLError, HTTPError

def fetch_with_headers(url, timeout=10):
    """
    Fetch a web page with proper headers and timeout.
    
    Args:
        url: The URL to fetch
        timeout: Max seconds to wait (default 10)
    
    Returns:
        Tuple of (content, status_code) or (None, error_code)
    """
    # Add a User-Agent to be polite and avoid blocks
    headers = {
        'User-Agent': 'Mozilla/5.0 (Educational Python Script)',
        'Accept': 'text/html'
    }
    
    try:
        request = Request(url, headers=headers)
        with urlopen(request, timeout=timeout) as response:
            content = response.read().decode('utf-8')
            return (content, response.status)
    except HTTPError as e:
        print(f"⚠️ HTTP Error {e.code}: {e.reason}")
        return (None, e.code)
    except URLError as e:
        print(f"❌ URL Error: {e.reason}")
        return (None, -1)
    except TimeoutError:
        print(f"⏰ Timeout after {timeout} seconds")
        return (None, -1)

# Demonstration (without actual network call)
print("✅ fetch_with_headers() function defined!")
print()
print("📝 Features added:")
print("  • Custom User-Agent header (identifies your bot)")
print("  • Accept header (tells server what we want)")
print("  • Timeout (prevents hanging)")
print("  • Returns status code for checking")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ✨ PYTHONIC: Complete web fetcher class with rate limiting
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Best: Rate limiting, retry logic, robots.txt checking

from urllib.request import urlopen, Request
from urllib.error import URLError, HTTPError
from urllib.parse import urlparse
import time

class WebFetcher:
    """
    A polite web fetcher with rate limiting and error handling.
    
    Features:
    - Rate limiting (respects servers)
    - Retry logic (handles temporary failures)
    - Custom headers (identifies your bot)
    - Timeout protection (prevents hanging)
    """
    
    def __init__(self, delay=1.0, retries=3, timeout=10):
        """
        Initialize the fetcher.
        
        Args:
            delay: Seconds between requests (be polite!)
            retries: Number of retry attempts
            timeout: Request timeout in seconds
        """
        self.delay = delay
        self.retries = retries
        self.timeout = timeout
        self.last_request_time = 0
        self.headers = {
            'User-Agent': 'EducationalBot/1.0 (Learning Python)',
            'Accept': 'text/html,application/xhtml+xml',
            'Accept-Language': 'en-US,en;q=0.9'
        }
    
    def _wait_for_rate_limit(self):
        """Wait if needed to respect rate limit."""
        elapsed = time.time() - self.last_request_time
        if elapsed < self.delay:
            wait_time = self.delay - elapsed
            print(f"⏳ Rate limiting: waiting {wait_time:.2f}s...")
            time.sleep(wait_time)
        self.last_request_time = time.time()
    
    def fetch(self, url):
        """
        Fetch a URL with retries and rate limiting.
        
        Returns:
            dict with 'content', 'status', 'success' keys
        """
        self._wait_for_rate_limit()
        
        for attempt in range(1, self.retries + 1):
            try:
                request = Request(url, headers=self.headers)
                with urlopen(request, timeout=self.timeout) as response:
                    content = response.read().decode('utf-8')
                    return {
                        'content': content,
                        'status': response.status,
                        'success': True,
                        'url': url
                    }
                    
            except HTTPError as e:
                print(f"  ⚠️ Attempt {attempt}: HTTP {e.code}")
                if e.code == 429:  # Rate limited
                    time.sleep(5 * attempt)  # Back off
                elif e.code >= 400:
                    return {'content': None, 'status': e.code, 'success': False, 'url': url}
                    
            except URLError as e:
                print(f"  ❌ Attempt {attempt}: {e.reason}")
                
            except Exception as e:
                print(f"  ❌ Attempt {attempt}: {e}")
            
            if attempt < self.retries:
                time.sleep(2 ** attempt)  # Exponential backoff
        
        return {'content': None, 'status': -1, 'success': False, 'url': url}

# Demonstration
fetcher = WebFetcher(delay=0.5, retries=2, timeout=5)
print("✅ WebFetcher class defined!")
print()
print("📊 Configuration:")
print(f"  • Delay between requests: {fetcher.delay}s")
print(f"  • Retry attempts: {fetcher.retries}")
print(f"  • Timeout: {fetcher.timeout}s")
print()
print("🎨 Usage example:")
print("  fetcher = WebFetcher()")
print('  result = fetcher.fetch("https://example.com")')
print('  if result["success"]:')
print('      html = result["content"]')

---

## 🔍 Section 3: Parsing HTML with html.parser

Now that we can fetch pages, let's extract data! The `html.parser` module uses an **event-driven** approach - you define handlers that get called when the parser encounters different HTML elements.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚫 BAD PRACTICE: Using string methods to parse HTML
# ═══════════════════════════════════════════════════════════════════════════════
# ❌ Problem: String methods break with complex HTML!

html = '''
<html>
<body>
    <a href="https://example.com">Example</a>
    <a href='https://test.com'>Test</a>
    <a 
       href="https://multiline.com">Multiline</a>
</body>
</html>
'''

# This approach is FRAGILE and will miss many links!
links = []
for line in html.split('\n'):
    if 'href="' in line:
        start = line.find('href="') + 6
        end = line.find('"', start)
        if end > start:
            links.append(line[start:end])

print("🚫 Links found with string parsing:")
print(links)
print()
print("⚠️ Problems with this approach:")
print("  ❌ Misses single-quoted attributes")
print("  ❌ Breaks with multiline attributes")
print("  ❌ Can't handle nested tags")
print("  ❌ No encoding handling")
print("  ❌ Regex would be even worse!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🌱 NOVICE: Basic HTMLParser for links
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Better: Properly parses HTML structure

from html.parser import HTMLParser

class SimpleLinkParser(HTMLParser):
    """A simple parser that extracts all links."""
    
    def __init__(self):
        super().__init__()
        self.links = []
    
    def handle_starttag(self, tag, attrs):
        """Called when parser finds an opening tag like <a>."""
        if tag == 'a':
            for name, value in attrs:
                if name == 'href':
                    self.links.append(value)

# Test HTML (same as before)
html = '''
<html>
<body>
    <a href="https://example.com">Example</a>
    <a href='https://test.com'>Test</a>
    <a 
       href="https://multiline.com">Multiline</a>
</body>
</html>
'''

# Parse it!
parser = SimpleLinkParser()
parser.feed(html)

print("✅ Links found with HTMLParser:")
for link in parser.links:
    print(f"  • {link}")

print()
print("💡 HTMLParser handles:")
print("  ✅ Single and double quotes")
print("  ✅ Multiline attributes")
print("  ✅ Proper HTML structure")
print("  ✅ Character encoding")

---

## 🎯 Section 4: Extracting Specific Data

Let's build more sophisticated parsers that can extract specific types of data.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 INTERMEDIATE: Parser for links with text content
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Better: Capture both URL and link text

from html.parser import HTMLParser

class LinkTextParser(HTMLParser):
    """Parser that extracts links with their text content."""
    
    def __init__(self):
        super().__init__()
        self.links = []         # List of (url, text) tuples
        self.current_link = None  # Currently parsing link
        self.current_text = ""    # Text inside current link
    
    def handle_starttag(self, tag, attrs):
        if tag == 'a':
            for name, value in attrs:
                if name == 'href':
                    self.current_link = value
                    self.current_text = ""
    
    def handle_data(self, data):
        """Called for text content between tags."""
        if self.current_link is not None:
            self.current_text += data.strip()
    
    def handle_endtag(self, tag):
        """Called for closing tags like </a>."""
        if tag == 'a' and self.current_link:
            self.links.append((self.current_link, self.current_text))
            self.current_link = None
            self.current_text = ""

# Sample HTML with links
html = '''
<html>
<body>
    <nav>
        <a href="/">Home</a>
        <a href="/about">About Us</a>
        <a href="/products">Our Products</a>
        <a href="https://external.com">External Link</a>
    </nav>
</body>
</html>
'''

parser = LinkTextParser()
parser.feed(html)

print("🔗 Links with Text Content:")
print("=" * 50)
for url, text in parser.links:
    link_type = "🔗 External" if url.startswith('http') else "📄 Internal"
    print(f"  {link_type} '{text}' → {url}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ✨ PYTHONIC: Complete data extractor with multiple elements
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Best: Extract headings, links, paragraphs, and images

from html.parser import HTMLParser
from urllib.parse import urljoin

class PageDataExtractor(HTMLParser):
    """
    Comprehensive HTML parser that extracts structured data.
    
    Extracts:
    - Page title
    - All headings (h1-h6)
    - All links with text
    - All images with alt text
    - All paragraphs
    """
    
    def __init__(self, base_url=""):
        super().__init__()
        self.base_url = base_url
        self.data = {
            'title': '',
            'headings': [],
            'links': [],
            'images': [],
            'paragraphs': []
        }
        
        # State tracking
        self._current_tag = None
        self._current_attrs = {}
        self._current_text = ""
    
    def handle_starttag(self, tag, attrs):
        attrs_dict = dict(attrs)
        self._current_tag = tag
        self._current_attrs = attrs_dict
        self._current_text = ""
        
        # Handle self-closing tags
        if tag == 'img':
            src = attrs_dict.get('src', '')
            alt = attrs_dict.get('alt', 'No alt text')
            if src:
                full_url = urljoin(self.base_url, src)
                self.data['images'].append({'src': full_url, 'alt': alt})
    
    def handle_data(self, data):
        if self._current_tag:
            self._current_text += data.strip()
    
    def handle_endtag(self, tag):
        text = self._current_text.strip()
        
        if tag == 'title' and text:
            self.data['title'] = text
            
        elif tag in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6'] and text:
            self.data['headings'].append({
                'level': int(tag[1]),
                'text': text
            })
            
        elif tag == 'a' and text:
            href = self._current_attrs.get('href', '')
            if href:
                full_url = urljoin(self.base_url, href)
                self.data['links'].append({
                    'url': full_url,
                    'text': text,
                    'external': full_url.startswith('http') and self.base_url not in full_url
                })
                
        elif tag == 'p' and text:
            self.data['paragraphs'].append(text)
        
        self._current_tag = None
        self._current_text = ""

# ═══════════════════════════════════════════════════════════════════════════════
# 📄 Sample Complex HTML Page
# ═══════════════════════════════════════════════════════════════════════════════
sample_page = '''
<!DOCTYPE html>
<html>
<head>
    <title>Python Tutorial Site</title>
</head>
<body>
    <h1>Welcome to Python Learning</h1>
    
    <nav>
        <a href="/">Home</a>
        <a href="/tutorials">Tutorials</a>
        <a href="https://python.org">Python.org</a>
    </nav>
    
    <main>
        <h2>Getting Started</h2>
        <p>Python is a versatile programming language perfect for beginners.</p>
        
        <img src="/images/python-logo.png" alt="Python Logo">
        
        <h2>Popular Topics</h2>
        <p>Learn about data types, functions, and classes.</p>
        
        <h3>Data Types</h3>
        <p>Strings, integers, floats, and more!</p>
    </main>
</body>
</html>
'''

# Parse the page
extractor = PageDataExtractor(base_url="https://example.com")
extractor.feed(sample_page)

# Display results
print("📄 Page Analysis Results")
print("=" * 60)

print(f"\n📌 Title: {extractor.data['title']}")

print(f"\n📑 Headings ({len(extractor.data['headings'])}):")
for h in extractor.data['headings']:
    indent = "  " * h['level']
    print(f"  {'#' * h['level']} {h['text']}")

print(f"\n🔗 Links ({len(extractor.data['links'])}):")
for link in extractor.data['links']:
    icon = "🌐" if link['external'] else "📄"
    print(f"  {icon} {link['text']} → {link['url']}")

print(f"\n🖼️ Images ({len(extractor.data['images'])}):")
for img in extractor.data['images']:
    print(f"  • {img['alt']} ({img['src']})")

print(f"\n📝 Paragraphs ({len(extractor.data['paragraphs'])}):")
for i, p in enumerate(extractor.data['paragraphs'], 1):
    preview = p[:50] + "..." if len(p) > 50 else p
    print(f"  {i}. {preview}")

---

## 🎯 Section 5: Practice Challenges

Try these exercises to reinforce your web scraping skills!

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎯 CHALLENGE 1: Extract Absolute URLs Only
# ═══════════════════════════════════════════════════════════════════════════════
# Task: Modify the LinkParser to only collect absolute URLs (starting with http)

html_challenge = '''
<html>
<body>
    <a href="https://google.com">Google</a>
    <a href="/local/page">Local Page</a>
    <a href="http://example.org">Example</a>
    <a href="../relative/path">Relative</a>
    <a href="https://python.org">Python</a>
</body>
</html>
'''

# TODO: Create a parser that only collects absolute URLs
# Expected output: ['https://google.com', 'http://example.org', 'https://python.org']

from html.parser import HTMLParser

class AbsoluteURLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.absolute_links = []
    
    def handle_starttag(self, tag, attrs):
        # TODO: Implement this!
        # Hint: Check if href starts with 'http'
        pass

# Your code here:
# parser = AbsoluteURLParser()
# parser.feed(html_challenge)
# print(parser.absolute_links)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ SOLUTION: Challenge 1 - Absolute URLs Only
# ═══════════════════════════════════════════════════════════════════════════════

from html.parser import HTMLParser

class AbsoluteURLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.absolute_links = []
    
    def handle_starttag(self, tag, attrs):
        if tag == 'a':
            for name, value in attrs:
                if name == 'href' and value.startswith(('http://', 'https://')):
                    self.absolute_links.append(value)

# Test it
html_challenge = '''
<html>
<body>
    <a href="https://google.com">Google</a>
    <a href="/local/page">Local Page</a>
    <a href="http://example.org">Example</a>
    <a href="../relative/path">Relative</a>
    <a href="https://python.org">Python</a>
</body>
</html>
'''

parser = AbsoluteURLParser()
parser.feed(html_challenge)

print("✅ Absolute URLs Found:")
for url in parser.absolute_links:
    print(f"  🌐 {url}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎯 CHALLENGE 2: Extract Product Data
# ═══════════════════════════════════════════════════════════════════════════════
# Task: Parse a product listing page and extract product info

product_html = '''
<html>
<body>
    <div class="product">
        <h2 class="name">Laptop Pro</h2>
        <span class="price">$999</span>
        <p class="description">High-performance laptop</p>
    </div>
    <div class="product">
        <h2 class="name">Wireless Mouse</h2>
        <span class="price">$29</span>
        <p class="description">Ergonomic design</p>
    </div>
    <div class="product">
        <h2 class="name">USB Keyboard</h2>
        <span class="price">$49</span>
        <p class="description">Mechanical keys</p>
    </div>
</body>
</html>
'''

# TODO: Create a parser that extracts:
# - Product names (from h2.name)
# - Prices (from span.price)
# - Descriptions (from p.description)
#
# Expected output: List of dicts with 'name', 'price', 'description'

# Hint: Track which class you're inside using handle_starttag attrs
# and use a state variable to know what type of data you're collecting

# Your code here:

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ SOLUTION: Challenge 2 - Product Data Extractor
# ═══════════════════════════════════════════════════════════════════════════════

from html.parser import HTMLParser

class ProductParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.products = []
        self.current_product = {}
        self.current_field = None
    
    def handle_starttag(self, tag, attrs):
        attrs_dict = dict(attrs)
        class_name = attrs_dict.get('class', '')
        
        if tag == 'div' and class_name == 'product':
            self.current_product = {}
        elif tag == 'h2' and class_name == 'name':
            self.current_field = 'name'
        elif tag == 'span' and class_name == 'price':
            self.current_field = 'price'
        elif tag == 'p' and class_name == 'description':
            self.current_field = 'description'
    
    def handle_data(self, data):
        if self.current_field and data.strip():
            self.current_product[self.current_field] = data.strip()
            self.current_field = None
    
    def handle_endtag(self, tag):
        if tag == 'div' and self.current_product:
            self.products.append(self.current_product)
            self.current_product = {}

# Test it
product_html = '''
<html>
<body>
    <div class="product">
        <h2 class="name">Laptop Pro</h2>
        <span class="price">$999</span>
        <p class="description">High-performance laptop</p>
    </div>
    <div class="product">
        <h2 class="name">Wireless Mouse</h2>
        <span class="price">$29</span>
        <p class="description">Ergonomic design</p>
    </div>
    <div class="product">
        <h2 class="name">USB Keyboard</h2>
        <span class="price">$49</span>
        <p class="description">Mechanical keys</p>
    </div>
</body>
</html>
'''

parser = ProductParser()
parser.feed(product_html)

print("🛒 Products Found:")
print("=" * 60)
for i, product in enumerate(parser.products, 1):
    print(f"\n📦 Product {i}:")
    print(f"   Name: {product.get('name', 'N/A')}")
    print(f"   Price: {product.get('price', 'N/A')}")
    print(f"   Description: {product.get('description', 'N/A')}")

---

## 📚 Session Recap

### What We Learned Today:

| Topic | Key Concept | Tool |
|-------|-------------|------|
| **HTTP Requests** | Fetching web pages | `urllib.request` |
| **Error Handling** | Try/except for network errors | `URLError`, `HTTPError` |
| **HTML Parsing** | Event-driven parsing | `html.parser.HTMLParser` |
| **Data Extraction** | Custom parser classes | `handle_starttag()`, `handle_data()` |
| **Best Practices** | Rate limiting, retries | Custom `WebFetcher` class |

### The Evolution of Our Code:

| Level | Characteristics |
|-------|-----------------|
| 🚫 **Bad** | No error handling, string parsing |
| 🌱 **Novice** | Basic try/except, simple parser |
| 🔧 **Intermediate** | Headers, timeout, link+text extraction |
| ✨ **Pythonic** | Rate limiting, retries, comprehensive extraction |

### Key HTMLParser Methods:

```python
class MyParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        # Called for <tag attr="value">
        pass
    
    def handle_data(self, data):
        # Called for text content
        pass
    
    def handle_endtag(self, tag):
        # Called for </tag>
        pass
```

### 🚨 Ethical Web Scraping Checklist:

- [ ] ✅ Check robots.txt before scraping
- [ ] ✅ Add delays between requests (1+ second)
- [ ] ✅ Identify your bot with User-Agent
- [ ] ✅ Handle errors gracefully
- [ ] ✅ Respect rate limits (429 status)
- [ ] ✅ Don't scrape personal data
- [ ] ✅ Use APIs when available

### 🎯 Next Steps:

- Try scraping a real (permitted) website
- Explore BeautifulSoup for easier parsing
- Learn about Selenium for JavaScript-heavy sites
- Practice with the group session activities!